# How to Build a `RuleBasedProfiler`
* This Notebook will demonstrate the steps we need to take to generate a simple `RuleBasedProfiler` by initializing the components in memory.

* We will start from a new Great Expectations Data Context (ie `great_expectations` folder after running `great_expectations init`), and begin by adding the Datasource, and progressively adding more components


In [1]:
import great_expectations as ge
from ruamel import yaml

from great_expectations.core.batch import BatchRequest
from great_expectations.core import ExpectationSuite

from great_expectations.rule_based_profiler.rule.rule import Rule
from great_expectations.rule_based_profiler.rule_based_profiler import RuleBasedProfiler

from great_expectations.rule_based_profiler.domain_builder import (
    DomainBuilder,
    ColumnDomainBuilder,
)
from great_expectations.rule_based_profiler.parameter_builder import (
    MetricMultiBatchParameterBuilder,
)
from great_expectations.rule_based_profiler.expectation_configuration_builder import (
    DefaultExpectationConfigurationBuilder,
)

In [2]:
data_context: ge.DataContext = ge.get_context()

## Set-up: Adding `taxi_data` `Datasource`
* Add `taxi_data` as a new `Datasource`
* We are using an `InferredAssetFilesystemDataConnector` to connect to data in the `test_sets/taxi_yellow_tripdata_samples` folder and get one `DataAsset` (`yellow_tripdata_sample_2018`) that has 12 Batches (1 Batch/month).

In [3]:
data_path: str = "../../../../test_sets/taxi_yellow_tripdata_samples"

datasource_config = {
    "name": "taxi_multi_batch_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine",
    },
    "data_connectors": {
        "default_inferred_data_connector_name": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "default_regex": {
                "group_names": ["data_asset_name", "month"],
                "pattern": "(yellow_tripdata_sample_2018)-(\\d.*)\\.csv",
            },
        },
        "default_inferred_data_connector_name_all_years": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "default_regex": {
                "group_names": ["data_asset_name", "year", "month"],
                "pattern": "(yellow_tripdata_sample)_(\\d.*)-(\\d.*)\\.csv",
            },
        },
    },
}

data_context.test_yaml_config(yaml.dump(datasource_config))


Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetFilesystemDataConnector

	Available data_asset_names (1 of 1):
		yellow_tripdata_sample_2018 (3 of 12): ['yellow_tripdata_sample_2018-01.csv', 'yellow_tripdata_sample_2018-02.csv', 'yellow_tripdata_sample_2018-03.csv']

	Unmatched data_references (3 of 29):['.DS_Store', 'first_3_files', 'random_subsamples']

	default_inferred_data_connector_name_all_years : InferredAssetFilesystemDataConnector

	Available data_asset_names (1 of 1):
		yellow_tripdata_sample (3 of 36): ['yellow_tripdata_sample_2018-01.csv', 'yellow_tripdata_sample_2018-02.csv', 'yellow_tripdata_sample_2018-03.csv']

	Unmatched data_references (3 of 5):['.DS_Store', 'first_3_files', 'random_subsamples']



In [4]:
# add_datasource only if it doesn't already exist in our configuration
try:
    data_context.get_datasource(datasource_config["name"])
except ValueError:
    data_context.add_datasource(**datasource_config)

# BatchRequests

* In this example, we will be using two `BatchRequests` using our `Datasource`. 
   * `single_batch_batch_request` : which gives the most recent (December) data from the 2018 `taxi_data` dataset. 
   * `multi_batch_batch_request`: which gives all 12 Batches of data from the 2018 `taxi_data` datataset.

In [5]:
single_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="yellow_tripdata_sample_2018",
    data_connector_query={"index": -1},
)

In [6]:
multi_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="yellow_tripdata_sample_2018",
)

# Example 1:  `RuleBasedProfiler` with just a `DomainBuilder` and `ExpectationConfigurationBuilder`

## Build a `DomainBuilder`

In the process of building a `RuleBasedProfiler`, one of the first components we want to build/test
is a `DomainBuilder`, which returns the Domains (tables, columns, set of columns, etc) that the our resulting `Expectations` will be run on. In our example, the `DomainBuilder` will output a list of columns that follow a certain pattern, namely have `'_amount'` in their suffix. To this end we will be using a `ColumnDomainBuilder` which allows you to choose columns based on their suffix, name, or semantic type (like numeric or string) and our `DomainBuilder` will output a list of 4 columns : `fare_amount`, `tip_amount`, `tolls_amount` and `total_amount`.

The `RuleBasedProfiler` also contains additional `DomainBuilders` that allow you to do more sophisticated filtering on your data. 

These include:
 * `CategoricalColumnDomainBuilder`: which allows you to choose columns based on their cardinality (number of unique values).
 * `MapMetricColumnDomainBuilder`: which allows you to choose columns based on Map Metrics, which give a yes/no answer for individual values or rows. 

In addition, there are `DomainBuilders` that do not perform any additional filtering, but are required by the Expectations that are being built by the `RuleBasedProfiler`. 
 * `TableDomainBuilder`:  Outputs Table Domain, which is required by `Expectations` that act on tables, like (`expect_table_row_count_to_equal`, or `expect_table_columns_to_match_set`). 


#### `ColumnDomainBuilder`

In [7]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    include_column_name_suffixes=["_amount"],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 108.38it/s]


In [8]:
# assert that the domains we get are the ones we expect
assert len(domains) == 4
assert domains == [
    {"domain_type": "column", "domain_kwargs": {"column": "fare_amount"}},
    {"domain_type": "column", "domain_kwargs": {"column": "tip_amount"}},
    {"domain_type": "column", "domain_kwargs": {"column": "tolls_amount"}},
    {"domain_type": "column", "domain_kwargs": {"column": "total_amount"}},
]

To continue our example, we will continue building a `RuleBasedProfiler` using our `ColumnDomainBuilder`

## Build `Rule`
* The first `Rule` that we build will output `expect_column_values_to_not_be_null` because it does not take in  additional information other than Domain. We will add `ParameterBuilders` in a subsequent example.

In [9]:
default_expectation_configuration_builder = DefaultExpectationConfigurationBuilder(
    expectation_type="expect_column_values_to_not_be_null",
    column="$domain.domain_kwargs.column", # Get the column from domain_kwargs that are retrieved from the DomainBuilder
)

In [10]:
simple_rule: Rule = Rule(
    name="rule_with_no_parameters",
    domain_builder=domain_builder,
    expectation_configuration_builders=[default_expectation_configuration_builder],
)

## Create `RuleBasedProfiler` and add `Rule`
* We create a simple `RuleBasedProfiler` and add the `Rule` that we added in the previous step is added to the Profiler. When we run the Profiler, the output is an `ExpectationSuite` with 4 `Expectations`, which we expect.

In [11]:
from great_expectations.core import ExpectationSuite
from great_expectations.rule_based_profiler.rule_based_profiler import RuleBasedProfiler

In [12]:
my_rbp: RuleBasedProfiler = RuleBasedProfiler(
    name="my_simple_rbp", data_context=data_context, config_version=1.0
)

In [13]:
my_rbp.add_rule(rule=simple_rule)


In [14]:
res: ExpectationSuite = my_rbp.run()

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 112.64it/s]


In [15]:
assert len(res.expectations) == 4

In [16]:
res.expectations

[{"kwargs": {"column": "fare_amount"}, "expectation_type": "expect_column_values_to_not_be_null", "meta": {}},
 {"kwargs": {"column": "tip_amount"}, "expectation_type": "expect_column_values_to_not_be_null", "meta": {}},
 {"kwargs": {"column": "tolls_amount"}, "expectation_type": "expect_column_values_to_not_be_null", "meta": {}},
 {"kwargs": {"column": "total_amount"}, "expectation_type": "expect_column_values_to_not_be_null", "meta": {}}]

As expected our simple `RuleBasedProfiler` will output an `ExpectationSuite` with 4 `Expectations`, one for each of our 4 columns. 

# Example 2: `RuleBasedProfiler` with `DomainBuilder`, `ParameterBuilder` `ExpectationConfigurationBuilder`

## Build a DomainBuilder
* Using the same `ColumnDomainBuilder` from our previous example.

In [17]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    include_column_name_suffixes=["_amount"],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 177.21it/s]


In [18]:
domains

[{
   "domain_type": "column",
   "domain_kwargs": {
     "column": "fare_amount"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "tip_amount"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "tolls_amount"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "total_amount"
   }
 }]

## Build a `ParameterBuilder`

`ParameterBuilders` help calcluate "reasonable" parameters for Expectations based on data that is specified by a `BatchRequest`.

The largest categories include: 
- `metric_multi_batch_parameter_builder`: Which is able to calculate a numeric Metric (like `column.min`) across multiple Batches (or just one Batch).
- `value_set_multi_batch_parameter_builder`: Which is able to build a value set across multiple Batches (or just one Batch). 

In some cases, there is a better way to build a value set using regex or dates. 
- `regex_pattern_string_parameter_builder`: Which contains a set of default regex patterns and builds a value set of the best-matching patterns. Users are also able to pass in new patterns as a parameter. 
- `simple_date_format_string_parameter_builder`: Which contains a set of default datetime-format patterns and builds a value set of the best-matching patterns. Users are also able to pass in new patterns as a parameter. 

Across multiple-Batches, we can build more-sophisticated parameters by using sampling methods. 
- `numeric_range_multi_batch_parameter_builder`: Which is able to provide range estimations across Batches using sampling methods. For instance, if we expect a table's `row_count` to change between Batches, we could calculate the min / max values of row_count by using the `NumericMetricRangeMultiBatchParameterBuilder`. These parameters could then be used by `ExpectTableRowCountToBeBetween`


In our example we will be using a `MetricMultiBatchParameterBuilder` to estimate the `column.min` Metric for the 4 columns defined by our Domain Builder. These are passed in as `metric_domain_kwargs` and are accessible using the fully qualified parameter `$domain.domain_kwargs`.

In [19]:
numeric_range_parameter_builder: MetricMultiBatchParameterBuilder = (
    MetricMultiBatchParameterBuilder(
        data_context=data_context,
        batch_request=single_batch_batch_request,
        metric_name="column.min",
        metric_domain_kwargs="$domain.domain_kwargs",  # domain kwarg values are accessible using fully qualified parameters
        name="my_column_min",
    )
)

## Build an ExpectationConfigurationBuilder

`ExpectationConfigurationBuilder` is being built for `expect_column_values_to_be_greater_than` which will use the `column.min` values that are calculated using the `ParameterBuilder`. These are now accessible using the fully qualified parameter `$parameter.my_column_min.value[-1]`.  The `[-1]` indicates that we will use the min value from the latest Batch (the only `Batch` in this case since our `BatchRequest` only returns a single `Batch`).

In [20]:
config_builder: DefaultExpectationConfigurationBuilder = (
    DefaultExpectationConfigurationBuilder(
        expectation_type="expect_column_values_to_be_greater_than",
        value="$parameter.my_column_min.value[-1]", # the parameter is accessible using a fully qualified parameter
        column="$domain.domain_kwargs.column", # domain kwarg values are accessible using fully qualified parameters
        name="my_column_min",
    )
)

## Build a `Rule`, `RuleBasedProfiler`, and run 

Now we build a rule with our `ParameterBuilder`, `DomainBuilder` and `ExpectationConfigurationBuilder`.

In [21]:
simple_rule: Rule = Rule(
    name="rule_with_parameters",
    domain_builder=domain_builder,
    parameter_builders=[numeric_range_parameter_builder],
    expectation_configuration_builders=[config_builder],
)

In [22]:
my_rbp = RuleBasedProfiler(name="my_rbp", data_context=data_context
                           , config_version=1.0)


Add the `Rule` to our `RuleBasedProfiler` and run. 

In [23]:
my_rbp.add_rule(rule=simple_rule)

In [24]:
res: ExpectationSuite = my_rbp.run()

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 147.09it/s]


In [25]:
assert len(res.expectations) == 4

In [26]:
res.expectations

[{"kwargs": {"value": -80.0, "name": "my_column_min", "column": "fare_amount"}, "expectation_type": "expect_column_values_to_be_greater_than", "meta": {}},
 {"kwargs": {"value": 0.0, "name": "my_column_min", "column": "tip_amount"}, "expectation_type": "expect_column_values_to_be_greater_than", "meta": {}},
 {"kwargs": {"value": 0.0, "name": "my_column_min", "column": "tolls_amount"}, "expectation_type": "expect_column_values_to_be_greater_than", "meta": {}},
 {"kwargs": {"value": -80.3, "name": "my_column_min", "column": "total_amount"}, "expectation_type": "expect_column_values_to_be_greater_than", "meta": {}}]

The resulting `ExpectationSuite` now contain values (`-80.0`, `0.0` etc) that were calculated from the Batch of data defined by the `BatchRequest`.

# Example 3: `RuleBasedProfiler` with multiple `ParameterBuilders`, `ExpectationConfigurationBuilders` and  `Variables`

The third example is more complex, using multiple batches, multiple `ParameterBuilders`, `ExpectationConfigurationBuilders` and also introducing the use of `variables`.

The goal of this example is to build a `RuleBasedProfiler` that outputs an `ExpectationSuite` containing 2 Expectation types
- `expect_column_min_to_be_between` : Defined as "Expect the column minimum to be between a min and max value".
- `expect_column_max_to_be_between` : Defined as "Expect the column maxmimum to be between a min and max value".

for 2 columns in our `taxi_data` dataset
- `fare_amount` 
- `tip_amount`

with the `min_value` and `max_value` parameters for each of the Expectations estimated over 12 Batches of `taxi_data`, for a total of 4 Expectations.

To estimate the parameters, we will be using a `NumericMetricRangeMultiBatchParameterBuilder`, which is able to provide range estimations across Batches using sampling methods.  We will also be using a `variables` dictionary to share defined variables across Rule components like `DomainBuilders`, `ParameterBuilders` and `ExpectationConfigurationBuilders`.

### Instantiating `variables` dictionary

`RuleBasedProfilers` allow for the definition of variables, which can be shared across Rules and Rule components. When building a complex `RuleBasedProfiler` with multiple Rules or components, using variables will help you keep track of values without having to input them multiple times.

Once loaded into the `RuleBasedProfiler` configuration, the variables are accessible using the fully qualified variable name `$variables.[key_in_variables_dictionary]`, similar to how domain kwarg values and parameter values are accessible using a fully qualified name that begins with `$`.

In the example below, the `estimator_name` is accessible using `$variables.estimator_name`.

In [27]:
variables: dict = {
    "multi_batch_batch_request": multi_batch_batch_request,
    "estimator_name": "bootstrap",
    "false_positive_rate": 5.0e-2, 
}

### Instantiating `RuleBasedProfiler` with `variables`

Pass the `variables` dictionary into the `RuleBasedProfiler` constructor. 

In [28]:
my_rbp = RuleBasedProfiler(name="my_complex_rbp", data_context=data_context, variables=variables, config_version=1.0)

### Instantiating `ColumnDomainBuilder`

The `ColumnDomainBuilder` is instantiated using column names `tip_amount` and `fare_amount`. The BatchRequest is passed in as a `$variable`. 

In [29]:
from great_expectations.rule_based_profiler.domain_builder import ColumnDomainBuilder

In [30]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    include_column_names=["tip_amount", "fare_amount"],
    batch_request="$variables.multi_batch_batch_request",
    data_context=data_context,
)

### Instantiating `ParameterBuilders`

Our Rule will contain 2 `NumericMetricRangeMultiBatchParameterBuilders`, one for each of our 2 Expectation types. One will be estimating the Parameter values for the `column.min` Metric, and the other will be estimating Parameter values for the `column.max` Metric.  `metric_domain_kwargs` are passed in from our DomainBuilder using `$domain.domain_kwargs`.

Also note the use of 3 Variables we defined above: 

- `$variables.estimator_name`: This is `"oneshot"` in our case. 
- `$variables.false_positive_rate`: This is `5.0e-2` or 5% in our case.
- `$variables.multi_batch_batch_request`: This the `multi_batch_batch_request`, which gives all 12 Batches of data from the 2018 taxi_data datataset.

In [31]:
from great_expectations.rule_based_profiler.parameter_builder import NumericMetricRangeMultiBatchParameterBuilder

In [32]:
min_range_parameter_builder: NumericMetricRangeMultiBatchParameterBuilder = NumericMetricRangeMultiBatchParameterBuilder(
    name="min_range_parameter_builder",
    metric_name="column.min",
    metric_domain_kwargs="$domain.domain_kwargs",
    false_positive_rate='$variables.false_positive_rate',
    estimator="$variables.estimator_name",
    batch_request="$variables.multi_batch_batch_request",
    data_context=data_context,
)

In [33]:
max_range_parameter_builder: NumericMetricRangeMultiBatchParameterBuilder = NumericMetricRangeMultiBatchParameterBuilder(
    name="max_range_parameter_builder",
    metric_name="column.max",
    metric_domain_kwargs="$domain.domain_kwargs",
    false_positive_rate="$variables.false_positive_rate",
    estimator="$variables.estimator_name",
    batch_request="$variables.multi_batch_batch_request",
    data_context=data_context,
)

### Instantiating `ExpectationConfigurationBuilders`

Our Rule will contain 2 `ExpectationConfigurationBuilders`, one for each of our 2 Expectation types: 

- `expect_column_min_to_be_between`
- `expect_column_max_to_be_between`

The Expectations are both `ColumnExpectations`, so the `column` parameter will be accessed from the Domain kwargs using `$domain.domain_kwargs.column`. 

The Expectations also take in a `min_value` and `max_value` parameter, which our `NumericMetricRangeMultiBatchParameterBuilders` are estimating. For `expect_column_min_to_be_between`, these estimated values are accessible using

- `$parameter.min_range_parameter_builder.value[0]` for the `min_value`, with `min_range_parameter_builder` being the name of our ParameterBuilder that estimates the `column.min` metric.
- `$parameter.min_range.value[1]` for the `max_value`.

The equivalent `$parameter` for `expect_column_max_to_be_between` would be `$parameter.max_range.value[0]` and `$parameter.max_range_parameter_builder.value[1]` respectively.

In [34]:
expect_column_min: DefaultExpectationConfigurationBuilder = DefaultExpectationConfigurationBuilder(
    expectation_type="expect_column_min_to_be_between",
    column="$domain.domain_kwargs.column",
    min_value="$parameter.min_range_parameter_builder.value[0]",
    max_value="$parameter.min_range_parameter_builder.value[1]",
)

In [35]:
expect_column_max: DefaultExpectationConfigurationBuilder = DefaultExpectationConfigurationBuilder(
    expectation_type="expect_column_max_to_be_between",
    column="$domain.domain_kwargs.column", 
    min_value="$parameter.max_range_parameter_builder.value[0]",
    max_value="$parameter.max_range_parameter_builder.value[1]",
)

### Instantiating `RuleBasedProfiler` and Running

We instantiate a Rule with our `DomainBuilder`, `ParameterBuilders` and `ExpectationConfigurationBuilders` and load into our `RuleBasedProfiler`.

In [36]:
more_complex_rule: Rule = Rule(
    name="rule_with_parameters",
    domain_builder=domain_builder,
    parameter_builders=[min_range_parameter_builder, max_range_parameter_builder],
    expectation_configuration_builders=[expect_column_min, expect_column_max],
)

In [37]:
my_rbp.add_rule(rule=more_complex_rule)

In [38]:
res: ExpectationSuite = my_rbp.run()

Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1243.85it/s]
/Users/work/Development/great_expectations/great_expectations/rule_based_profiler/helpers/util.py:579: RuntimeWarning: invalid value encountered in double_scalars
  if bootstrap_upper_quantile_bias / bootstrap_upper_quantile_standard_error <= 0.25:
Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1446.18it/s]


In [39]:
res.expectations

[{"kwargs": {"min_value": -1.66025, "max_value": 0.0, "column": "tip_amount"}, "expectation_type": "expect_column_min_to_be_between", "meta": {}},
 {"kwargs": {"min_value": 34.4595, "max_value": 156.172786779, "column": "tip_amount"}, "expectation_type": "expect_column_max_to_be_between", "meta": {}},
 {"kwargs": {"min_value": -94.5, "max_value": -4.309350435, "column": "fare_amount"}, "expectation_type": "expect_column_min_to_be_between", "meta": {}},
 {"kwargs": {"min_value": 167.375, "max_value": 235038.99560331, "column": "fare_amount"}, "expectation_type": "expect_column_max_to_be_between", "meta": {}}]

As expected, the resulting ExpectationSuite contains our minimum and maximum values, with `tip_amount` ranging from `$-2.16` to `$195.05` (a generous tip), and `fare_amount` ranging from `$-98.90` (a refund) to `$405,904.54` (a very very long trip).


# Appendix

* Here we have additional example configuration of `DomainBuilder` and `ParameterBuilders` that were not included in the previous 3 Examples. 

## `DomainBuilders`

#### `ColumnDomainBuilder`
This `DomainBuilder` outputs column Domains, which are required by `ColumnExpectations` like (`expect_column_median_to_be_between`). There are a few ways that the `ColumnDomainBuilder` can be used. 

1. In the simplest usecase, the `ColumnDomainBuilder` can output all columns in the dataset as a Domain, or include/exclude columns if you already know which ones you would like. Column suffixes (like `_amount`) can be used to select columns of interest, as we saw in our examples above.
3. The `ColumnDomainBuilder` also allows you to choose columns based on their semantic types (such as numeric, or text).

    - Semantic types are defined as an Enum object called SemanticDomainTypes, which can be found here : https://github.com/great-expectations/great_expectations/blob/develop/great_expectations/rule_based_profiler/types/domain.py


In [40]:
from great_expectations.rule_based_profiler.domain_builder import ColumnDomainBuilder

In the simplest usecase, the `ColumnDomainBuilder` can output all of the columns in `yellow_tripdata_sample_2018`

In [41]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
assert len(domains) == 18 # all columns in yellow_tripdata_sample_2018

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 119.86it/s]


Columns can also be included or excluded by name

In [42]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    include_column_names=["vendor_id"],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
domains

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 100.27it/s]


[{
   "domain_type": "column",
   "domain_kwargs": {
     "column": "vendor_id"
   }
 }]

In [43]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    exclude_column_names=["vendor_id"],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
assert len(domains) == 17 # all columns in yellow_tripdata_sample_2018 with vendor_id excluded

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 133.96it/s]


In [44]:
domains

[{
   "domain_type": "column",
   "domain_kwargs": {
     "column": "pickup_datetime"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "dropoff_datetime"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "passenger_count"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "trip_distance"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "rate_code_id"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "store_and_fwd_flag"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "pickup_location_id"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "dropoff_location_id"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "payment_type"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": {
     "column": "fare_amount"
   }
 },
 {
   "domain_type": "column",
   "domain_kwargs": 

As described above, the `ColumnDomainBuilder` also allows you to choose columns based on their semantic types (such as numeric, or text). This is passed in as part of the `include_semantic_types` parameter.

In [45]:
domain_builder: DomainBuilder = ColumnDomainBuilder(
    include_semantic_types=['numeric'],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)

In [46]:
domains: list = domain_builder.get_domains()
assert len(domains) == 15 # columns in yellow_trip_data_sample_2018 that are numeric

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 128.09it/s]


**`MultiColumnDomainBuilder`**

This DomainBuilder outputs `multicolumn` Domains by taking in a column list in the `include_column_names` parameter.

In [47]:
from great_expectations.rule_based_profiler.domain_builder import MultiColumnDomainBuilder

In [48]:
domain_builder: DomainBuilder = MultiColumnDomainBuilder(
    include_column_names=["vendor_id", "fare_amount", "tip_amount"],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
assert len(domains) == 1 # 3 columns are part of a single multi-column domain. 

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 139.42it/s]


In [49]:
expected_columns: list = ["vendor_id", "fare_amount", "tip_amount"]

In [50]:
assert domains[0]["domain_kwargs"]["column_list"] == expected_columns

**ColumnPairDomainBuilder**

This DomainBuilder outputs `ColumnPair` Domains by taking in a column pair list in the `include_column_names` parameter.

In [51]:
from great_expectations.rule_based_profiler.domain_builder import ColumnPairDomainBuilder

In [52]:
domain_builder: DomainBuilder = ColumnPairDomainBuilder(
    include_column_names=["vendor_id", "fare_amount"],
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
assert len(domains) == 1 # 3 columns are part of a single multi-column domain. 

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 120.92it/s]


In [53]:
expect_columns_dict: dict = {'column_A': 'fare_amount', 'column_B': 'vendor_id'}

In [54]:
assert domains[0]["domain_kwargs"] == expect_columns_dict

#### `TableDomainBuilder`
This `DomainBuilder` outputs table `Domains`, which is required by `Expectations` that act on tables, like (`expect_table_row_count_to_equal`, or `expect_table_columns_to_match_set`).

In [55]:
from great_expectations.rule_based_profiler.domain_builder import TableDomainBuilder

In [56]:
domain_builder: DomainBuilder = TableDomainBuilder(
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
domains

[{
   "domain_type": "table"
 }]

#### `MapMetricColumnDomainBuilder`

This `DomainBuilder` allows you to choose columns based on Map Metrics, which give a yes/no answer for individual values or rows. In this example, we use the Map Metrics `column_values.nonnull` to filter out a column that was all `None` from `taxi_data`. 

In [57]:
from great_expectations.rule_based_profiler.domain_builder import MapMetricColumnDomainBuilder

In [58]:
domain_builder: DomainBuilder = MapMetricColumnDomainBuilder(
    map_metric_name="column_values.nonnull",
    batch_request=single_batch_batch_request,
    data_context=data_context,
)
domains: list = domain_builder.get_domains()
len(domains) == 17 # filtered 1 column that was all None

Calculating Metrics: 100%|█████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 424.15it/s]


True

#### `CategoricalColumnDomainBuilder`

This `DomainBuilder` allows you to choose columns based on their cardinality (number of unique values).The `CategoricalColumnDomainBuilder` will take in various `limit_modes` for cardinality, and in this example we are only interested in columns that have "very_few" (less than 10) unique values. For a full of valid modes, along with the associated values, please refer to the `CardinalityLimitMode` enum in:

https://github.com/great-expectations/great_expectations/blob/develop/great_expectations/rule_based_profiler/helpers/cardinality_checker.py

In [59]:
from great_expectations.rule_based_profiler.domain_builder import CategoricalColumnDomainBuilder

In [60]:
domain_builder: DomainBuilder = CategoricalColumnDomainBuilder(
    limit_mode="very_few", # VERY_FEW = 10 or less
    batch_request=single_batch_batch_request,
    data_context=data_context,
)

In [61]:
domains: list = domain_builder.get_domains()
assert len(domains) == 9

Calculating Metrics: 100%|█████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 314.77it/s]


## `ParameterBuilders`

`ParameterBuilders` work under the hood by populating a `ParameterContainer`, which can also be shared by multiple `ParameterBuilders`. It requires a Domain, and `metric_name`, with `domain_kwargs` accessible from the `DomainBuilder` using the fully qualified parameter `$domain.domain_kwargs`.

For the sake of simplicity, we will define a `Domain` object directly using the `Domain()` constructor, and pass in a column name within `domain_kwargs`. 

In [62]:
from great_expectations.rule_based_profiler.types.domain import Domain
from great_expectations.execution_engine.execution_engine import MetricDomainTypes
from great_expectations.rule_based_profiler.types import ParameterContainer

In [63]:
domain: Domain = Domain(domain_type=MetricDomainTypes.COLUMN, domain_kwargs = {'column': 'total_amount'})

#### `MetricMultiBatchParameterBuilder`

The `MetricMultiBatchParameterBuilder` computes a Metric on data from one or more batches. It takes `domain_kwargs`, `value_kwargs`, and `metric_name` as arguments.

In [64]:
from great_expectations.rule_based_profiler.parameter_builder import MetricMultiBatchParameterBuilder

In [65]:
numeric_range_parameter_builder: MetricMultiBatchParameterBuilder = (
    MetricMultiBatchParameterBuilder(
        data_context=data_context,
        batch_request=multi_batch_batch_request, # we are passing in our multi_batch_batch_request here
        metric_name="column.min",
        metric_domain_kwargs=domain.domain_kwargs,
        name="my_column_min",
    )
)

In [66]:
parameter_container: ParameterContainer = ParameterContainer(parameter_nodes=None)

In [67]:
numeric_range_parameter_builder.build_parameters(domain=domain, parameter_container=parameter_container)
# we check the parameter container
print(parameter_container.parameter_nodes)


Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1206.21it/s]

{'parameter': {'parameter': {'my_column_min': {'value': [-19.8, -57.3, -6.8, -63.06, -11.8, -6.8, -30.6, -16.8, -4.3, -100.8, -12.8, -80.3], 'details': {'metric_configuration': {'metric_name': 'column.min', 'domain_kwargs': {'column': 'total_amount'}, 'metric_value_kwargs': None, 'metric_dependencies': None}, 'num_batches': 12}}}}}


In [68]:
min(parameter_container.parameter_nodes["parameter"]["parameter"]["my_column_min"]["value"])

-100.8

`my_column_min[value]` now contains a list of 12 values, which are the minimum values the `total_amount` column for each of the 12 Batches associated with 2018 `taxi_data` data. If we were to use the values in a `ExpectationConfigurationBuilder`, it would be accessible through the fully-qualified parameter: `$parameter.my_column_min.value`.

#### `ValueSetMultiBatchParameterBuilder`

The `ValueSetMultiBatchParameterBuilder` is able to build a value set across multiple Batches (or just one Batch). 

In [69]:
from great_expectations.rule_based_profiler.parameter_builder import ValueSetMultiBatchParameterBuilder

In [70]:
domain: Domain = Domain(domain_type=MetricDomainTypes.COLUMN, domain_kwargs = {'column': 'vendor_id'})

In [71]:
# instantiating a new parameter container, since it can contain the results of more than one ParmeterBuilder. 
parameter_container: ParameterContainer = ParameterContainer(parameter_nodes=None)

In [72]:
value_set_parameter_builder: ValueSetMultiBatchParameterBuilder = (
    ValueSetMultiBatchParameterBuilder(
        data_context=data_context,
        batch_request= multi_batch_batch_request,
        metric_domain_kwargs=domain.domain_kwargs,
        name="my_value_set",
    )
)

In [73]:
value_set_parameter_builder.build_parameters(
    parameter_container=parameter_container,
    domain=domain,
    parameters = {domain.id: parameter_container}
)

Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1089.32it/s]


In [74]:
print(parameter_container.parameter_nodes)

{'parameter': {'parameter': {'my_value_set': {'value': [1, 2, 4], 'details': {'metric_configuration': {'metric_name': 'column.distinct_values', 'domain_kwargs': {'column': 'vendor_id'}, 'metric_value_kwargs': None, 'metric_dependencies': None}, 'num_batches': 12}}}}}


`my_value_set[value]` now contains a list of 3 values, which is a list of all unique `vendor_ids` across 12 Batches in the 2018 `taxi_data` dataset.

#### `RegexPatternStringParameterBuilder`

The `RegexPatternStringParameterBuilder` contains a set of default regex patterns and builds a value set of the best-matching patterns. Users are also able to pass in new patterns as a parameter.

In [75]:
from great_expectations.rule_based_profiler.parameter_builder import RegexPatternStringParameterBuilder

In [76]:
domain: Domain = Domain(domain_type=MetricDomainTypes.COLUMN, domain_kwargs = {'column': 'vendor_id'})

* `vendor_id` is a single integer. Let's see if our default patterns can match it. 

In [77]:
parameter_container: ParameterContainer = ParameterContainer(parameter_nodes=None)

In [78]:
regex_parameter_builder: RegexPatternStringParameterBuilder = (
    RegexPatternStringParameterBuilder(
        data_context=data_context,
        batch_request=single_batch_batch_request,
        metric_domain_kwargs=domain.domain_kwargs,
        name="my_regex_set",
    )
)

In [79]:
regex_parameter_builder.build_parameters(
    parameter_container=parameter_container,
    domain=domain,
    parameters = {domain.id: parameter_container}
)

Calculating Metrics:   0%|                                                                                           | 0/25 [00:00<?, ?it/s]/Users/work/Development/great_expectations/great_expectations/expectations/metrics/column_map_metrics/column_values_match_regex.py:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return column.astype(str).str.contains(regex)
Calculating Metrics: 100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 73.17it/s]


In [80]:
print(parameter_container.parameter_nodes)

{'parameter': {'parameter': {'my_regex_set': {'value': None, 'details': {'success_ratio': 0.0, 'evaluated_regexes': {'/<\\/?(?:p|a|b|img)(?: \\/)?>/': 0.0, '/https?:\\/\\/(www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{2,256}\\.[a-z]{2,6}\\b([-a-zA-Z0-9@:%_\\+.~#()?&//=]*)/': 0.0, '\\s+/$': 0.0, '/(?:[A-Fa-f0-9]){0,4}(?: ?:? ?(?:[A-Fa-f0-9]){0,4}){0,7}/': 0.0, '/-?\\d+(\\.\\d*)?/': 0.0, '/(?:25[0-5]|2[0-4]\\d|[01]\\d{2}|\\d{1,2})(?:.(?:25[0-5]|2[0-4]\\d|[01]\\d{2}|\\d{1,2})){3}/': 0.0, '\\b[0-9a-fA-F]{8}\\b-[0-9a-fA-F]{4}-[0-5][0-9a-fA-F]{3}-[089ab][0-9a-fA-F]{3}-\\b[0-9a-fA-F]{12}\\b ': 0.0, '/[A-Za-z0-9\\.,;:!?()\\"\'%\\-]+/': 0.0, '/-?\\d+/': 0.0, '/\\d+/': 0.0, '^\\s+/': 0.0}}}}}}


* Looks like `my_regex_set[value]` is an empty list. This means that none of the `evaluated_regexes` matched our domain. Let's try the same thing again, but this time with a regex that will match our `vendor_id` column. `^\\d{1}$` and `^\\d{2}$` which will match 1 or 2 digit integers anchored at the beginning and end of the string.

In [81]:
regex_parameter_builder: RegexPatternStringParameterBuilder = (
    RegexPatternStringParameterBuilder(
        data_context=data_context,
        batch_request=single_batch_batch_request,
        metric_domain_kwargs=domain.domain_kwargs,
        candidate_regexes=["^\\d{1}$"],
        name="my_regex_set",
    )
)

In [82]:
regex_parameter_builder.build_parameters(
    parameter_container=parameter_container,
    domain=domain,
    parameters = {domain.id: parameter_container}
)

Calculating Metrics: 100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 144.65it/s]


In [83]:
print(parameter_container.parameter_nodes)

{'parameter': {'parameter': {'my_regex_set': {'value': '^\\d{1}$', 'details': {'success_ratio': 1.0, 'evaluated_regexes': {'^\\d{1}$': 1.0}}}}}}


* Now `my_regex_set[value]` contains `^\\d{1}$`.

#### `SimpleDateFormatStringParameterBuilder`

The `SimpleDateFormatStringParameterBuilder` contains a set of default Datetime format patterns and builds a value set of the best-matching patterns. Users are also able to pass in new patterns as a parameter.

In [84]:
from great_expectations.rule_based_profiler.parameter_builder import SimpleDateFormatStringParameterBuilder

In [85]:
domain: Domain = Domain(domain_type=MetricDomainTypes.COLUMN, domain_kwargs = {'column': 'pickup_datetime'})

In [86]:
parameter_container: ParameterContainer = ParameterContainer(parameter_nodes=None)

In [87]:
simple_date_format_string_parameter_builder: SimpleDateFormatStringParameterBuilder = (
    SimpleDateFormatStringParameterBuilder(
        data_context=data_context,
        batch_request=single_batch_batch_request,
        metric_domain_kwargs=domain.domain_kwargs,
        name="my_value_set",
    )
)

In [88]:
simple_date_format_string_parameter_builder.build_parameters(
    parameter_container=parameter_container,
    domain=domain,
    parameters = {domain.id: parameter_container}
)

Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:02<00:00, 45.24it/s]


In [89]:
print(parameter_container.parameter_nodes)

{'parameter': {'parameter': {'my_value_set': {'value': '%Y-%m-%d %H:%M:%S', 'details': {'success_ratio': 1.0, 'candidate_strings': {'%Y-%m-%d %H:%M:%S': 1.0, '%H:%M:%S.%f': 0.0, '%Y-%m-%d %H:%M:%S%z': 0.0, '%Y-%m-%d %H:%M:%S %z': 0.0, '%Y %b %d %H:%M:%S.%f %Z': 0.0, '%H:%M:%S,%f': 0.0, '%m/%d/%Y %H:%M:%S %p': 0.0, '%m/%d/%Y*%H:%M:%S': 0.0, '%b %d %Y %H:%M:%S': 0.0, '%Y-%m-%d %H:%M:%S,%f%z': 0.0, "%Y-%m-%d'T'%H:%M:%S%z": 0.0, '%Y-%m-%d %H:%M:%S,%f': 0.0, '%Y/%m/%d*%H:%M:%S': 0.0, '%b %d %H:%M:%S': 0.0, '%y/%m/%d %H:%M:%S': 0.0, '%Y-%m-%d*%H:%M:%S:%f': 0.0, '%Y/%m/%d': 0.0, '%y%m%d %H:%M:%S': 0.0, '%b %d, %Y %H:%M:%S %p': 0.0, '%m/%d/%Y*%H:%M:%S*%f': 0.0, '%b %d %H:%M:%S %Y': 0.0, '%m/%d/%y*%H:%M:%S': 0.0, "%Y-%m-%d'T'%H:%M:%S": 0.0, '%d/%b/%Y:%H:%M:%S': 0.0, '%b %d %H:%M:%S %z': 0.0, "%Y-%m-%d'T'%H:%M:%S.%f": 0.0, '%d/%b/%Y:%H:%M:%S %z': 0.0, '%Y-%m-%d*%H:%M:%S': 0.0, '%m/%d/%Y': 0.0, '%y-%m-%d %H:%M:%S': 0.0, '%Y%m%d %H:%M:%S.%f': 0.0, '%y/%m/%d': 0.0, '%m%d_%H:%M:%S': 0.0, "%Y-%m-%d'T

In [90]:
parameter_container.parameter_nodes["parameter"]["parameter"]["my_value_set"]["value"]

'%Y-%m-%d %H:%M:%S'

The result contains our matching `datetime` pattern, which is `'%Y-%m-%d %H:%M:%S'`

#### `NumericMetricRangeMultiBatchParameterBuilder`

The `NumericMetricRangeMultiBatchParameterBuilder` is able to provide range estimations across Batches using sampling methods. For instance, if we expect a table's row_count to change between Batches, we could calculate the min / max values of row_count by using the `NumericMetricRangeMultiBatchParameterBuilder`. These parameters could then be used by `Expectations` that take in ranges, like `ExpectTableRowCountToBeBetween`, or `ExpectColumnValuesToBeBetween`.

In this example, we will be taking a single Metric, `column.mean` and calculating it for a single column, `total_amount`. The parameter we will be building is the column mean-range, which are the min-max values of the `total_amount` column across random samples of 12 Batches of the 2018 `taxi_data` dataaset. 

We will also be passing in specifications for estimator, namely `bootstrap` sampling with a false-positive rate of less than 0.01. 

In [91]:
from great_expectations.rule_based_profiler.parameter_builder import NumericMetricRangeMultiBatchParameterBuilder

In [92]:
domain: Domain = Domain(domain_type=MetricDomainTypes.COLUMN, domain_kwargs = {'column': 'total_amount'})

In [93]:
numeric_metric_range_parameter_builder: NumericMetricRangeMultiBatchParameterBuilder = NumericMetricRangeMultiBatchParameterBuilder(
    name="column_mean_range",
    metric_name="column.mean",
    estimator="bootstrap",
    metric_domain_kwargs=domain.domain_kwargs,
    false_positive_rate=1.0e-2,
    round_decimals=0,
    data_context=data_context,
    batch_request=multi_batch_batch_request,
)

In [94]:
parameter_container: ParameterContainer = ParameterContainer(parameter_nodes=None)

In [95]:
numeric_metric_range_parameter_builder.build_parameters(
    parameter_container=parameter_container,
    domain=domain,
    parameters = {domain.id: parameter_container}
)

Calculating Metrics: 100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 1516.30it/s]


In [96]:
print(parameter_container.parameter_nodes)

{'parameter': {'parameter': {'column_mean_range': {'value': [16.0, 44.0], 'details': {'metric_configuration': {'metric_name': 'column.mean', 'domain_kwargs': {'column': 'total_amount'}, 'metric_value_kwargs': None, 'metric_dependencies': None}, 'num_batches': 12}}}}}


As we see, the mean value range for the `total_amount` column is `16.0` to `44.0`

## Optional: Clean-up Directory


As part of running this notebook, the `RuleBasedProfiler` will create a number of ExpectationSuite configurations in the `great_expectations/expectations/tmp` directory. Optionally run the following cell to clean up the directory.

In [101]:
#import shutil, os
#shutil.rmtree("great_expectations/expectations/tmp")
#os.remove("great_expectations/expectations/.ge_store_backend_id")